## PD结合的思路

### 分别替换gate/up层

先输入一句话，按照70%稀疏去记录prefill阶段激活的神经元，最后统计这个输入prompt对应的最高激活次数的70%的神经元。

In [1]:
import os
import json
import torch
from transformers import LlamaForCausalLM, AutoTokenizer
import convert_llama
from transformers import GenerationConfig
from datasets import load_dataset

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

### from path.json read paths of model and dataset
model_name = "Llama3-8b"
dataset_name = "c4"
with open('path.json', 'r') as file:
    paths = json.load(file)
    model_path = paths.get(model_name, '')
    dataset_path = paths.get(dataset_name, '')

c4 = load_dataset(dataset_path)
model = LlamaForCausalLM.from_pretrained(
    model_path,
    device_map='auto',
    use_cache=True,
    torch_dtype=torch.float16,
)
convert_llama.convert_llama_model(model, sparsity=0.1, start_num=14, end_num=16, )

tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/home/lz/anaconda3/envs/moe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.03s/it]
Convert Llama Models: 452it [00:00, 167742.47it/s]


Converted Model Done


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
for c4_demo in c4['validation']['text'][:1]:
    input_demo = tokenizer(c4_demo, padding="max_length", truncation=True, max_length=200, return_tensors="pt")
    generate_ids = model.generate(input_demo.input_ids.to('cuda:0'), max_length=230, generation_config=GenerationConfig(do_sample=False), pad_token_id=tokenizer.eos_token_id)
    tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    model.model.layers[15].mlp.gate_proj.coreinfer_recall()
    model.model.layers[15].mlp.up_proj.coreinfer_recall()


[prefill] in gate layer: 15
[prefill] in up layer: 15
in decode, gate layer 15
Overlap count: 1243.8621, Overlap ratio: 0.8680
in decode, up layer 15
Overlap count: 1080.8966, Overlap ratio: 0.7543


In [2]:
### 使用greedy decode
generate_ids = model.generate(input_demo.input_ids.to('cuda:0'), max_length=230, generation_config=GenerationConfig(do_sample=False), pad_token_id=tokenizer.eos_token_id)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

[prefill] in gate layer: 15
[prefill] in up layer: 15


'The woman who died after falling from a bridge over the A21 has been identified as a Sevenoaks mum.\nMarta Kendle, 37, fell from the Gracious Lane bridge on the morning of February 19.\nPolice were called to the carriageway around 6.10am and the road was promptly closed in both directions.\nDespite paramedics best efforts, Marta, who was originally from Poland, was pronounced dead at the scene.\nKent and Medway Coroners office have confirmed an inquest into her death will open on Wednesday (February 27).\nTributes to the mum were left at the scene and on social media.\nFriend, Jodi Cahill posted on Facebook: "I will certainly remember you. I am sorry we did not see how lost and alone you felt.\n"Be at peace dear Marta."\nA floral tribute left at the scene said goodbye to the "beautiful and kind soul".\nIt read: "To a beautiful and kind soul. You will be missed. Rest in peace."\nA spokesman for Kent Police said: "Officers were called to the A21 at Gracious Lane,'

In [3]:
model.model.layers[15].mlp.gate_proj.coreinfer_recall()
model.model.layers[15].mlp.up_proj.coreinfer_recall()

in decode, gate layer 15
Overlap count: 1243.8621, Overlap ratio: 0.8680
in decode, up layer 15
Overlap count: 1080.8966, Overlap ratio: 0.7543
